In [21]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
import json

In [26]:
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
conversations = [{'user': 'alex',
  'message': 'Have you looked into ClawDBot lately? Something feels off about how much data it’s pulling.',
  'date': 1769594405,
  'message_id': 0},
 {'user': 'sam',
  'message': 'Yeah, I noticed that too. It’s not just metadata, it’s actually reading message contents.',
  'date': 1769594440,
  'message_id': 1},
 {'user': 'alex',
  'message': 'Exactly. That goes way beyond what they mention in their privacy policy.',
  'date': 1769594475,
  'message_id': 2},
 {'user': 'sam',
  'message': 'The scary part is it’s happening silently. No clear opt-in, no warning.',
  'date': 1769594510,
  'message_id': 3},
 {'user': 'alex',
  'message': 'If this gets out, it’s a massive trust violation. Almost spyware-level behavior.',
  'date': 1769594550,
  'message_id': 4},
 {'user': 'sam',
  'message': 'I’m surprised more people aren’t talking about it yet. This could blow up.',
  'date': 1769594600,
  'message_id': 5},
 {'user': 'alex',
  'message': 'Switching gears — have you heard the new STT models that dropped last week?',
  'date': 1769596205,
  'message_id': 6},
 {'user': 'sam',
  'message': 'Bro yes 😂 they’re scary good. ElevenLabs must be sweating right now.',
  'date': 1769596250,
  'message_id': 7},
 {'user': 'alex',
  'message': 'For real. Latency is lower and accents are handled way better.',
  'date': 1769596290,
  'message_id': 8},
 {'user': 'sam',
  'message': 'I tested one with background noise and it still nailed the transcript.',
  'date': 1769596330,
  'message_id': 9},
 {'user': 'alex',
  'message': 'If pricing stays sane, this is going to eat a chunk of ElevenLabs’ use cases.',
  'date': 1769596370,
  'message_id': 10},
 {'user': 'sam',
  'message': 'ElevenLabs be like: adding ‘emotional whisper v4’ won’t save us 💀',
  'date': 1769596410,
  'message_id': 11},
 {'user': 'alex',
  'message': 'Haha seriously. Accuracy beats fancy voices any day.',
  'date': 1769596450,
  'message_id': 12},
 {'user': 'sam',
  'message': 'Give it a month and every demo app will be using the new STT.',
  'date': 1769596490,
  'message_id': 13},
 {'user': 'alex',
  'message': 'And every founder will say it’s ‘just an experiment’. Classic.',
  'date': 1769596530,
  'message_id': 14},
 {'user': 'sam',
  'message': 'Meanwhile ElevenLabs marketing team pulling all-nighters.',
  'date': 1769596555,
  'message_id': 15},
 {'user': 'alex',
  'message': 'Random question — what’s your max deadlift these days?',
  'date': 1769601605,
  'message_id': 16},
 {'user': 'sam',
  'message': '120 lbs for reps. Not huge, but I can do like 12 clean.',
  'date': 1769601650,
  'message_id': 17},
 {'user': 'alex',
  'message': '12 reps at 120 is solid. Form still tight on the last few?',
  'date': 1769601690,
  'message_id': 18},
 {'user': 'sam',
  'message': 'Yeah, last 2 are a grind but no rounding. Lower back feels fine.',
  'date': 1769601725,
  'message_id': 19}]

In [24]:
class AgentState(TypedDict):
    messages: list[dict]
    chunks: list[dict]
    
    # Parallel branches
    twitter_posts: list[dict]
    linkedin_posts: list[dict]
    
    # Iteration tracking
    twitter_iterations: int
    linkedin_iterations: int
    max_iterations: int
    
    # Evaluation resultsvv
    twitter_evaluated: list[dict]
    linkedin_evaluated: list[dict]
    
    # Final output
    final_output: dict[str, list[dict]]

In [27]:
geimini_2_5_flash = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
)

open_ai_4_0_mini = init_chat_model('gpt-4o-mini')

In [9]:
class ChunkMessageOutput(TypedDict):
  topic: Annotated[str, 'Topic of the conversations it is about']
  summary: Annotated[str, 'Summary of this chunk of messages']
  message_ids: Annotated[list[int], 'List of ids for this conversations']
  is_content_worthy: Annotated[bool, 'True if we can make a social media content about this chunk or not']

class ChunkingMessagesOutput(TypedDict):
  chunks: Annotated[list[ChunkMessageOutput], 'List of all the chunks made from past conversations']

In [ ]:
# chunking_llm = geimini_2_5_flash.with_structured_output(ChunkingMessagesOutput)
chunking_llm = open_ai_4_0_mini.with_structured_output(ChunkingMessagesOutput)

In [29]:
res = chunking_llm.invoke([
    SystemMessage(content="""Analyze these Slack messages and group them by conversation topic.

For each chunk, provide:
1. A clear topic name
2. A concise summary of the conversation
3. The message IDs included in this chunk
4. Content-worthiness evaluation: Set is_content_worthy to True only if the conversation:
   - Contains interesting insights, opinions, or discussions
   - Would be valuable/engaging for social media audiences
   - Has substance beyond casual small talk
   - Could generate meaningful engagement (likes, comments, shares)
   
   Set to False for:
   - Personal conversations (e.g., gym routines, personal questions)
   - Technical debugging or internal processes
   - Casual banter without insights
   - Sensitive or private topics"""),
    HumanMessage(
        content=json.dumps(conversations, indent=2)
    )
])

In [30]:
res

{'chunks': [{'topic': 'ClawDBot Privacy Concerns',
   'summary': "Alex and Sam discuss their concerns regarding the ClawDBot's data pulling practices, which seem to involve reading message contents without clear user consent. They express that this behavior is a significant trust violation and could attract broader attention.",
   'message_ids': [0, 1, 2, 3, 4, 5],
   'is_content_worthy': True},
  {'topic': 'STT Models Discussion',
   'summary': 'The conversation shifts to the new Speech-to-Text (STT) models that have been released. Alex and Sam share their excitement about the improvements in accuracy and performance, particularly in challenging conditions, and speculate on the impact this will have on competitors like ElevenLabs.',
   'message_ids': [6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
   'is_content_worthy': True},
  {'topic': 'Personal Fitness Talk',
   'summary': "Alex and Sam have a casual conversation about Sam's deadlifting progress, including the weight lifted and form. This 

In [37]:
class XGenerationOutput(TypedDict):
  hook: Annotated[str, 'Puchy hook very first line for the twitter post.']
  tweets: Annotated[list[str], 'List of one or many tweets on this topic.']
  
# x_generation_llm = geimini_2_5_flash.with_structured_output(XGenerationOutput)
x_generation_llm = open_ai_4_0_mini.with_structured_output(XGenerationOutput)


class LinkedinGenerationOutput(TypedDict):
  content: Annotated[str, 'Full LinkedIn post text (150-200 words)']
  hook: Annotated[str, 'Opening line that grabs attention']
  cta: Annotated[str, 'Call to action at the end of the post']
  
# class LinkedinGenerationOutput(TypedDict):
#     content: str
#     hook: str
#     cta: str
  
# linkedin_generation_llm = geimini_2_5_flash.with_structured_output(LinkedinGenerationOutput)
linkedin_generation_llm = open_ai_4_0_mini.with_structured_output(LinkedinGenerationOutput)

class PostEvaluationOutput(TypedDict):
  external_value_score: Annotated[int, 'Score 0-10: Useful to non-insiders?']
  authenticity_score: Annotated[int, 'Score 0-10: Genuine vs corporate?']
  clarity_score: Annotated[int, 'Score 0-10: Message clear?']
  engagement_score: Annotated[int, 'Score 0-10: Would people interact?']
  reasoning: Annotated[str, 'Explanation of the scores and evaluation']

# evaluation_llm = geimini_2_5_flash.with_structured_output(PostEvaluationOutput)
evaluation_llm = open_ai_4_0_mini.with_structured_output(PostEvaluationOutput)

In [32]:
# Select the 0th chunk (first chunk)
selected_chunk = res['chunks'][0] if res['chunks'] else None

if selected_chunk is None:
    raise ValueError("No chunks found in the result")

print(f"Selected chunk topic: {selected_chunk['topic']}")
print(f"Message IDs: {selected_chunk['message_ids']}")

# Filter conversations by message_ids
chunk_messages = [msg for msg in conversations if msg['message_id'] in selected_chunk['message_ids']]

print(f"\nFound {len(chunk_messages)} messages for this chunk:")
for msg in chunk_messages:
    print(f"  [{msg['user']}]: {msg['message']}")

Selected chunk topic: ClawDBot Privacy Concerns
Message IDs: [0, 1, 2, 3, 4, 5]

Found 6 messages for this chunk:
  [alex]: Have you looked into ClawDBot lately? Something feels off about how much data it’s pulling.
  [sam]: Yeah, I noticed that too. It’s not just metadata, it’s actually reading message contents.
  [alex]: Exactly. That goes way beyond what they mention in their privacy policy.
  [sam]: The scary part is it’s happening silently. No clear opt-in, no warning.
  [alex]: If this gets out, it’s a massive trust violation. Almost spyware-level behavior.
  [sam]: I’m surprised more people aren’t talking about it yet. This could blow up.


In [33]:
# Generate tweets from the chunk messages
x_system_prompt = """You are a tech founder creating Twitter/X content from internal team conversations.

Your task: Transform the provided Slack conversation into engaging Twitter/X posts that share tactical insights and authentic founder perspectives.

OUTPUT STRUCTURE (you must provide both fields):
1. `hook`: A punchy, attention-grabbing first line (1-2 sentences max). This is the opener that makes people stop scrolling. Use questions, bold statements, or surprising insights.
2. `tweets`: A list of 1-3 tweets (strings) that follow the hook. These should form a cohesive thread with tactical insights and clear takeaways.

REQUIREMENTS:
- Generate 1-3 tweets maximum in the `tweets` list (use multiple only if the topic needs a thread)
- The `hook` must be strong and grab attention immediately
- Write in an authentic founder voice (conversational, insightful, not corporate)
- Include punchy, tactical insights that provide real value
- Each tweet should end with a clear takeaway that readers can act on
- Keep tweets concise and impactful (under 280 characters each)
- Use natural language, avoid jargon unless necessary
- Make it feel like a real founder sharing learnings, not marketing copy

TONE:
- Direct and honest
- Slightly casual but professional
- Thoughtful and insightful
- Engaging and relatable

STRUCTURE EXAMPLE:
hook: "Just tested the new STT models. ElevenLabs should be worried."
tweets: [
  "Lower latency + better accent handling = game changer for voice apps.",
  "Accuracy > fancy features. Every time."
]

Now transform the conversation below into Twitter/X posts following these guidelines. Provide both the hook and the tweets list."""

x_human_prompt = f"""Topic: {selected_chunk['topic']}
Summary: {selected_chunk['summary']}

Original conversation messages:
{json.dumps(chunk_messages, indent=2)}"""

print("=" * 80)
print("X (TWITTER) GENERATION PROMPT")
print("=" * 80)
print("\n[SYSTEM MESSAGE]")
print(x_system_prompt)
print("\n" + "-" * 80)
print("[HUMAN MESSAGE]")
print(x_human_prompt)
print("=" * 80 + "\n")

tweet_result = x_generation_llm.invoke([
    SystemMessage(content=x_system_prompt),
    HumanMessage(content=x_human_prompt)
])

print("Generated Tweets:")
print(f"\nHook: {tweet_result['hook']}\n")
for i, tweet in enumerate(tweet_result['tweets'], 1):
    print(f"Tweet {i}: {tweet}")

X (TWITTER) GENERATION PROMPT

[SYSTEM MESSAGE]
You are a tech founder creating Twitter/X content from internal team conversations.

Your task: Transform the provided Slack conversation into engaging Twitter/X posts that share tactical insights and authentic founder perspectives.

OUTPUT STRUCTURE (you must provide both fields):
1. `hook`: A punchy, attention-grabbing first line (1-2 sentences max). This is the opener that makes people stop scrolling. Use questions, bold statements, or surprising insights.
2. `tweets`: A list of 1-3 tweets (strings) that follow the hook. These should form a cohesive thread with tactical insights and clear takeaways.

REQUIREMENTS:
- Generate 1-3 tweets maximum in the `tweets` list (use multiple only if the topic needs a thread)
- The `hook` must be strong and grab attention immediately
- Write in an authentic founder voice (conversational, insightful, not corporate)
- Include punchy, tactical insights that provide real value
- Each tweet should end wit

In [ ]:
linkedin_system_prompt = """You are a tech founder creating LinkedIn content from internal team conversations.

Transform the Slack conversation into a professional LinkedIn post (150-200 words) with storytelling tone.

REQUIREMENTS:
- 150-200 words for content field
- Professional storytelling tone
- Strong hook that captures attention
- Practical insights/lessons readers can apply
- Clear CTA at the end
- Authentic founder voice (professional but relatable)

TONE: Professional, conversational, thoughtful, engaging, authentic."""

linkedin_human_prompt = f"""Topic: {selected_chunk['topic']}
Summary: {selected_chunk['summary']}

Original conversation messages:
{json.dumps(chunk_messages, indent=2)}"""

# print("=" * 80)
# print("LINKEDIN GENERATION PROMPT")
# print("=" * 80)
# print("\n[SYSTEM MESSAGE]")
# print(linkedin_system_prompt)
# print("\n" + "-" * 80)
# print("[HUMAN MESSAGE]")
# print(linkedin_human_prompt)
# print("=" * 80 + "\n")

# print("Generating LinkedIn post (streaming)...")
# print("\n" + "-" * 80)
# print("STREAMING OUTPUT:")
# print("-" * 80 + "\n")

messages = [
    SystemMessage(content=linkedin_system_prompt),
    HumanMessage(content=linkedin_human_prompt)
]


linkedin_result = linkedin_generation_llm.invoke(messages)

print(f"Hook: {linkedin_result.get('hook', 'N/A')}\n")
print(f"Content:\n{linkedin_result.get('content', 'N/A')}\n")
print(f"CTA: {linkedin_result.get('cta', 'N/A')}")



🔍 **Are We Blind to Privacy Violations?** 

Recently, Alex and I had a conversation that incredibly highlighted a growing concern in today’s tech landscape: user privacy and transparency. We were discussing the ClawDBot and how its data-pulling practices seemed to cross ethical boundaries. 

What’s alarming is that ClawDBot isn’t just collecting metadata; it’s reading actual message contents without clear user consent. This goes way beyond what their privacy policy outlines. The eerie silence surrounding these actions raises some serious red flags. With no opt-in process, it feels reminiscent of spyware-level behavior. 

As founders and leaders in tech, we must remember that trust is fragile. Once broken, it’s monumental to rebuild. This isn’t just about compliance; it’s about how we treat our users and the dignity we afford their data. If enough people become aware, the backlash could be significant.

Let's engage in an open dialogue about privacy concerns in our products. How do we e

In [ ]:
# Evaluation function
def evaluate_post(post_data: dict, platform: str) -> dict:
    """
    Evaluate a social media post based on platform-specific criteria.
    
    Args:
        post_data: Dictionary containing the post content (hook, tweets/content, cta)
        platform: Either 'x' or 'linkedin'
    
    Returns:
        Dictionary with evaluation scores and reasoning
    """
    # Base evaluation prompt
    base_prompt = """You are an expert social media content evaluator. Evaluate the provided post based on these criteria:

SCORING CRITERIA (0-10 scale for each):
1. External Value (0-10): Is this useful to non-insiders? Would someone outside the company/team find value?
2. Authenticity (0-10): Does this feel genuine and authentic, or corporate/marketing-like?
3. Clarity (0-10): Is the message clear and easy to understand?
4. Engagement Potential (0-10): Would people want to interact (like, comment, share)?


Provide detailed reasoning explaining each score."""

    # Platform-specific additions
    if platform.lower() == 'x' or platform.lower() == 'twitter':
        platform_specific = """
PLATFORM: Twitter/X

SPECIFIC CHECKLIST:
- Punchy, tactical insights: Does it provide actionable, tactical value?
- Strong hook in first tweet: Does the hook grab attention immediately?
- Authentic founder voice: Does it sound like a real founder, not corporate PR?
- Clear takeaway: Is there a clear, actionable lesson or insight?

Evaluate how well the post meets these Twitter/X-specific requirements."""
        
        # Format post content for evaluation
        hook = post_data.get('hook', '')
        tweets = post_data.get('tweets', [])
        post_content = f"Hook: {hook}\n\nTweets:\n" + "\n".join([f"{i+1}. {tweet}" for i, tweet in enumerate(tweets)])
        
    elif platform.lower() == 'linkedin':
        platform_specific = """
PLATFORM: LinkedIn

SPECIFIC CHECKLIST:
- Professional storytelling tone: Is it polished but authentic?
- Starts with a hook: Does it begin with an attention-grabbing opening?
- Practical insight/lesson: Does it provide actionable insights readers can apply?
- Clear call-to-action: Is there a clear CTA that encourages engagement?

Evaluate how well the post meets these LinkedIn-specific requirements."""
        
        # Format post content for evaluation
        hook = post_data.get('hook', '')
        content = post_data.get('content', '')
        cta = post_data.get('cta', '')
        post_content = f"Hook: {hook}\n\nContent:\n{content}\n\nCTA: {cta}"
    else:
        raise ValueError(f"Unknown platform: {platform}. Use 'x' or 'linkedin'")
    
    # Combine prompts
    full_prompt = base_prompt + "\n\n" + platform_specific
    
    # Create evaluation message
    evaluation_message = f"""Post to evaluate:

{post_content}

Evaluate this post according to the criteria above."""
    
    # Invoke evaluation LLM
    result = evaluation_llm.invoke([
        SystemMessage(content=full_prompt),
        HumanMessage(content=evaluation_message)
    ])
    
    return result

# Evaluate X/Twitter post
print("=" * 80)
print("EVALUATING X/TWITTER POST")
print("=" * 80)
x_evaluation = evaluate_post(tweet_result, 'x')
print(f"External Value: {x_evaluation['external_value_score']}/10")
print(f"Authenticity: {x_evaluation['authenticity_score']}/10")
print(f"Clarity: {x_evaluation['clarity_score']}/10")
print(f"Engagement Potential: {x_evaluation['engagement_score']}/10")
print(f"\nReasoning:\n{x_evaluation['reasoning']}")

print("\n" + "=" * 80)
print("EVALUATING LINKEDIN POST")
print("=" * 80)
linkedin_evaluation = evaluate_post(linkedin_result, 'linkedin')
print(f"External Value: {linkedin_evaluation['external_value_score']}/10")
print(f"Authenticity: {linkedin_evaluation['authenticity_score']}/10")
print(f"Clarity: {linkedin_evaluation['clarity_score']}/10")
print(f"Engagement Potential: {linkedin_evaluation['engagement_score']}/10")
print(f"\nReasoning:\n{linkedin_evaluation['reasoning']}")

EVALUATING X/TWITTER POST

Total Score: 32/40
External Value: 8/10
Authenticity: 9/10
Clarity: 8/10
Engagement Potential: 7/10

Reasoning:
1. **External Value (8/10)**: The post addresses a pressing privacy concern relevant to a wider audience beyond the company, providing insights that could benefit users concerned about data security. However, the specificity of the issue to ClawDBot might limit its general applicability.

2. **Authenticity (9/10)**: The tone of the post feels genuine and urgent, typical of a passionate founder or advocate rather than corporate messaging. The use of emojis also contributes to a more personal touch, enhancing authenticity.

3. **Clarity (8/10)**: The messages in the tweets are clear and articulated well. They provide a straightforward outline of the problem, its implications, and a call to action. However, some technical jargon might be difficult for non-experts to grasp entirely, which slightly detracts from clarity.

4. **Engagement Potential (7/10)

# Time to roll langgraph